In [1]:
include("load_s1slc_ann.jl");
include("load_pod.jl");
include("calc_sat_trajectory.jl");

In [2]:
f = open("ann_path.txt")
path = readlines(f)
meta = load_s1slc_ann(path[1]);
f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],meta["t_0"]);

In [3]:
f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],meta["t_0"]);

In [4]:
t_start = meta["t_start"]
t_stop = meta["t_stop"]
time_test = 53.01648932401438

53.01648932401438

In [5]:
osv_poly, osv_mean, osv_std = calc_sat_trajectory(osv, t_sv, t_start, t_stop);
osv_test = polyval_sv(osv_poly,time_test,osv_mean, osv_std);

In [6]:
include("approx_los.jl");

In [8]:
approx_los(osv_test,meta)

3-element Array{Float64,1}:
 -0.356271677770493 
 -0.7999912912283155
 -0.4827876191745395

 ### correct los
 -0.39871976313527907
 -0.7301475699415844 
 -0.554893752527226

In [11]:
theta = meta["incidence_angle_mid"]*pi/180

0.7608462959597286

In [19]:
using LinearAlgebra

In [20]:
theta_0 = meta["incidence_angle_mid"]*pi/180

x_sat = osv_test[1:3]
v_sat = osv_test[4:6]

losSat = [-cos(theta_0), sin(theta_0), 0]

# SAT1 basis in ECEF coordinates
x_hat_ecef = x_sat / sqrt(x_sat'*x_sat);
z_hat_ecef = v_sat / sqrt(v_sat'*v_sat);
y_hat_ecef = cross(z_hat_ecef, x_hat_ecef);

In [26]:
m = hcat(x_hat_ecef, y_hat_ecef,z_hat_ecef)
Ulos = m*losSat;

In [27]:
Ulos

3-element Array{Float64,1}:
 -0.356271677770493 
 -0.7999912912283155
 -0.4827876191745395

In [28]:
## Correct  Ulos

 -0.39871976313527907
 -0.7301475699415844 
 -0.554893752527226

In [10]:
using LinearAlgebra
function approx_los(osv_fit,s1_annn)
    theta_0 = s1_annn["incidence_angle_mid"]*pi/180

    x_sat = osv_fit[1:3]
    v_sat = osv_fit[4:6]

    #ECEF basis coordinates
    x_hat_ecef = x_sat / sqrt(x_sat'*x_sat) # Towards earth center
    z_hat_ecef = v_sat / sqrt(v_sat'*v_sat) # flight direction
    y_hat_ecef = cross(z_hat_ecef, x_hat_ecef) # Right handed coordinate system
    
    # Line of sight ECEF basis
    losSat = [-cos(theta_0), sin(theta_0), 0]
    
    # Basis change matrix from ECEF basis to elipsidal coordinates
    m = hcat(x_hat_ecef, y_hat_ecef,z_hat_ecef) 
    
    # Line of sight in Ellipsoidal coordinates
    los = m*losSat;

    return los
end


approx_los (generic function with 1 method)

In [11]:
approx_los(osv_test,meta)

3-element Array{Float64,1}:
 -0.356271677770493 
 -0.7999912912283155
 -0.4827876191745395

In [ ]:
range, zeroDopplerTime, osv, s1_annn

In [15]:
sqrt(osv_test[1:3]'*osv_test[1:3])

7.067937079742404e6

In [ ]:
losSat = [-cos(theta0) sin(theta0) 0]'; 
% SAT1 basis in ECEF coordinates
Xhat_ecef = XSat / norm(XSat);
Zhat_ecef = VSat / norm(VSat);
Yhat_ecef = cross(Zhat_ecef, Xhat_ecef);